## Sieć dla agenta do gry w Connect4 

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline

import torch 
import torch.nn as nn
import torch.nn.functional as F

from torchsummary import summary

from imp import reload

import DataLoader
reload(DataLoader)

from DataLoader import InMemDataLoader
from DataLoader import C4DataSet

/tmp/ipykernel_39592/4128569561.py:10: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
/home/michal/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import wandb
wandb.login() # klucz - 7242fe50822869937a282970b5385963778c7f8c

/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.2.1) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.2.1) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
wandb: Currently logged in as: michalfica125 (neuralnetworks). Use `wandb login --relogin` to force relogin


True

In [3]:
class Model(nn.Module):
    def __init__(self, dp=0.5):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(2, 25, 2, padding=1)
        self.bn1 = nn.BatchNorm2d(25)

        self.conv2 = nn.Conv2d(25, 15, 2, padding=1)
        self.bn2 = nn.BatchNorm2d(15)

        self.fc1 = nn.Linear(15*2*2, 12)
        self.bn3 = nn.BatchNorm1d(12)

        self.fc2 = nn.Linear(12, 3)

        self.do = nn.Dropout(dp)

    def forward(self, x):

        x = F.max_pool2d(self.conv1(x), 2)
        x = F.relu(self.bn1(x))

        x = F.max_pool2d(self.conv2(x), 2)
        x = F.relu(self.bn2(x))

        x = x.view(x.shape[0], -1)

        x = self.fc1(x)
        x = F.relu(self.bn3(x))

        x = self.do(x) # dropout

        x = self.fc2(x)

        x = nn.Softmax()(x)

        return x

    def loss(self, Out, Targets):
      return F.cross_entropy(Out, Targets)

model = Model()
summary(model, (2, 6, 8))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1             [-1, 25, 7, 9]             225
       BatchNorm2d-2             [-1, 25, 3, 4]              50
            Conv2d-3             [-1, 15, 4, 5]           1,515
       BatchNorm2d-4             [-1, 15, 2, 2]              30
            Linear-5                   [-1, 12]             732
       BatchNorm1d-6                   [-1, 12]              24
           Dropout-7                   [-1, 12]               0
            Linear-8                    [-1, 3]              39
Total params: 2,615
Trainable params: 2,615
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.02
Params size (MB): 0.01
Estimated Total Size (MB): 0.03
----------------------------------------------------------------


/home/michal/.local/lib/python3.10/site-packages/torch/nn/modules/module.py:1511: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
amount_of_games = 10000 
moves_observed  = 15 
all_samples = amount_of_games * moves_observed

batch_size = 128
train_size, val_size, test_size = int(all_samples/3), int(all_samples/3), int(all_samples/3)
amount_of_train_batches = train_size / batch_size

dataset = C4DataSet(amount_of_games, moves_observed).create_data_set()

train_set = dataset[:train_size]
val_set = dataset[train_size:train_size+val_size] 
test_set = dataset[train_size+val_size:]

data_loaders = {
    "train": InMemDataLoader(train_set, batch_size=batch_size, shuffle=True),
    "valid": InMemDataLoader(val_set, batch_size=batch_size, shuffle=False),
    "test": InMemDataLoader(test_set, batch_size=batch_size, shuffle=False),
}

  0%|          | 0/50000 [00:00<?, ?it/s]/home/michal/Documents/studia2/2sem/neutral-networks/assigmnent4/DataLoader.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch = [torch.tensor(t) for t in dataset[i]]
100%|██████████| 50000/50000 [00:00<00:00, 76466.81it/s]


In [6]:
def compute_error_rate(model, data_loader, device="cpu"):
  model.eval()

  num_errs, num_examples = 0, 0
  with torch.no_grad():
    for batch in data_loader:

      x, y = batch[0].to(device), batch[1].to(device)
      out = model(x)

      _, pred = out.max(dim=1)
      num_errs += (pred != y.data).sum().item()
      num_examples += x.size(0)

  return num_errs / num_examples

In [16]:
def train(num_of_epochs, train_loader, opt, print_every=10, device="cpu"):
  model.train()

  for data_loader in data_loaders.values():
    if isinstance(data_loader, InMemDataLoader):
        data_loader.to(device)

  min_batch_err = 100

  iter = 0
  for e in range(num_of_epochs):
    model.train()
    print(f"Epoch {e+1}")

    for batch in train_loader:
      x = batch[0].to(device)
      y = batch[1].to(device)
      opt.zero_grad()
      iter += 1

      out = model(x)
      loss = nn.CrossEntropyLoss()(out, y)
      loss.backward()
      opt.step()

      _, pred = out.max(dim=1)
      batch_err = (pred != y).sum().item() / out.size(0)
      min_batch_err = min(min_batch_err, batch_err)

      if iter % print_every == 0:
        print(f"iter = {iter}, batch_err = {batch_err * 100.0}")
        wandb.log({"batch_error_rate": batch_err})

    val_err = compute_error_rate(model, data_loader=data_loaders["valid"], device=device)
    print(f"val err = {100*val_err:.2f}")
    wandb.log({'val_err': val_err, 'epoch': e+1})

In [8]:
def initialize_weights(model):
    with torch.no_grad():
        for name, p in model.named_parameters():
            if 'weight' in name:
                if 'conv' in name:
                    f_in = p.shape[1]*p.shape[2]*p.shape[3]
                    p.normal_(0, torch.sqrt(torch.tensor(2./f_in)))
                elif 'bn' in name:
                    p = torch.ones_like(p)
                elif 'fc' in name:
                    f_in = p.shape[1]
                    p.normal_(0, torch.sqrt(torch.tensor(2./f_in)))
                else:
                    raise Exception('weird weight')

            elif 'bias' in name:
                p.zero_()
            else:
                raise Exception('weird parameter')

In [9]:
cnt = 0

In [10]:
lr = 0.00002
weight_decay = 0.000001
momentum = 0.9
epochs = 10
_device = "cpu"
_print_every = 30

# opt = torch.optim.SGD(model.parameters(), lr=lr, weight_decay = weight_decay, momentum = momentum)
opt = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

In [17]:
model = Model().to(device)
initialize_weights(model)

cnt +=  1
run_name = "run_nr " + str(cnt) + " 10 000 games" 
run = wandb.init(
    project="Assigmnent4",
    name = run_name,
    config={
        "epochs": epochs,
        "learning_rate": lr,
        "optimizer": "ADAM",
        "momentum": momentum,
        "weight_decay": weight_decay,
        "batch & dropout": True,
    },
)
wandb.watch(model, F.cross_entropy, log="all")

train(num_of_epochs=epochs, train_loader=data_loaders["train"], opt=opt, print_every=_print_every, device=_device)
test_error_rate = compute_error_rate(model, data_loaders["test"], device=_device)
wandb.finish()

Epoch 1
torch.Size([128, 2, 6, 8])


/home/michal/.local/lib/python3.10/site-packages/torch/nn/modules/module.py:1511: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


In [28]:
print(train_set[0][0])
with torch.no_grad():
    x = torch.tensor(train_set[0][0])
    print(f"x.shape = {x.shape}")
    x = x[None, :, :, :]
    print(f"x.shape = {x.shape}")
    out = model(x)
    print(out)
    print(out[0][1].item())

tensor([[[0., 0., 0., 0., 0., 1., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 1., 0., 1., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 1., 1., 0.],
         [0., 1., 0., 1., 0., 1., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0., 1., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0.],
         [0., 1., 0., 0., 0., 1., 0., 0.],
         [0., 0., 0., 1., 0., 0., 0., 0.],
         [1., 0., 1., 0., 0., 0., 1., 0.]]])
x.shape = torch.Size([2, 6, 8])
x.shape = torch.Size([1, 2, 6, 8])
tensor([[0.1944, 0.4376, 0.3680]])
0.4375842809677124


/tmp/ipykernel_39592/812249990.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(train_set[0][0])
/home/michal/.local/lib/python3.10/site-packages/torch/nn/modules/module.py:1511: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
